In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


data = pd.read_csv("output_file.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann


In [3]:
import numpy as np
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define SOSA parameters
num_spiders = 30
max_iterations = 100
alpha = 0.1  # Control the movement intensity
beta = 0.1   # Control the attraction/repulsion intensity

# Initialize spiders (particles)
spiders = np.random.uniform(-1, 1, (num_spiders, X_train.shape[1]))

# Define the fitness function
def fitness_function(spider):
    ann = create_ann()
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = np.mean((predictions - y_test) ** 2)
    return mse

# Main SOSA loop with progress bar
for iteration in tqdm(range(max_iterations), desc="SOSA Training"):
    fitness_values = np.array([fitness_function(spider) for spider in spiders])
    
    # Determine the best and worst fitness values
    best_fitness = np.min(fitness_values)
    worst_fitness = np.max(fitness_values)
    best_spider = spiders[np.argmin(fitness_values)]
    
    # Update each spider's position
    for i in range(num_spiders):
        if np.random.rand() < 0.8:  # Simulating social interaction probability
            if fitness_values[i] > best_fitness:
                direction = best_spider - spiders[i]
            else:
                random_spider = spiders[np.random.randint(0, num_spiders)]
                direction = random_spider - spiders[i]
            spiders[i] = spiders[i] + alpha * direction * np.random.rand()
        else:  # Simulating random movement
            spiders[i] = spiders[i] + beta * (2 * np.random.rand(X_train.shape[1]) - 1)
        
        # Ensure the spiders' positions are within bounds
        spiders[i] = np.clip(spiders[i], -1, 1)

# Select the best spider
best_spider_idx = np.argmin(fitness_values)
best_spider = spiders[best_spider_idx]

# Train the final ANN with the best parameters
final_ann = create_ann()
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = np.mean((final_predictions - y_test) ** 2)
print("Final MSE:", final_mse)


SOSA Training: 100%|██████████| 100/100 [02:02<00:00,  1.22s/it]

Final MSE: 47316.720591619414
